# 协程
- 参考资料
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/1190000009781688
    
# 迭代器
- 可迭代的(iterable): 直接作用于for循环的变量
- 迭代器(iterator): 不但可以作用于for循环，还可以被next调用
- list是一个典型的可迭代对象，但不是迭代器！！！
- 通过isinstance判断
- iterable和iterator可以转换
    - 通过iter函数

In [2]:
# 可迭代
l = [i for i in range(10)] 

# l是可迭代的，但不是迭代器
for idx in l:
    print(idx)
    
# range是一个迭代器
for i in range(5):
    print(i)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [6]:
# isinstance案例
# 判断某个变量是否是一个实例

# 判断是否可迭代
from collections import Iterable
ll = [1,2,3,4,5]

print(isinstance(ll, Iterable))

from collections import Iterator
print(isinstance(ll, Iterator))

True
False


In [8]:
# iter函数
from collections import Iterable, Iterator
s = "I love wangerqing"
print(isinstance(s, Iterable))
print(isinstance(s, Iterator))
      
s_iter = iter(s)
print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))


True
False
True
True


# 生成器
- generator：一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件;
    - 每次调用都产生出for训话需要的下一个元素
    - 如果达到最后一个后，报出StopIteration异常
    - 可以被next函数调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield，则这个函数就叫生成器
    - next调用函数，遇到yield返回

In [10]:
# 直接使用生成器

L = [i*i for i in range(5)] # 放在中括号中是列表生成器
g = (i*i for i in range(5)) # 放在小括号中就是生成器

print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [11]:
# 函数案例
def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None

odd()

Step 1
Step 2
Step 3


In [17]:
# 生成器的案例
# 在函数odd中，yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
    
# odd() 是调用生成器
g = odd()
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [22]:
# for 循环调用生成器
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        print(b)
        a, b = b, a + b # 注意写法
        n = n + 1
    return "Done"

print(fib(5))

1
1
2
3
5
Done


In [24]:
# 斐波那契数列的生成器写法
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        yield b
        a, b = b, a + b # 注意写法
        n = n + 1
    # 需要注意，报出异常的返回值是return的返回值
    return "Done"

g = fib(5)
for i in range(6):
    rst = next(g)
    print(rst)
    

1
1
2
3
5


StopIteration: Done

In [25]:
ge = fib(10)
'''
生成器的典型用法是在for中使用
比较常用的典型生成器就是range
'''
for i in ge: # 在for循环中使用生成器
    print(i)
    

1
1
2
3
5
8
13
21
34
55


# 协程
- 历史进程
    - 3.4引入协程，用field实现
    - 3.5引入协程语法
    - 实现协程比较好的包有asyncio，tornado，gevent
- 定义：
    - 协程：是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序
    - 从技术角度上讲，协程就是一个你可以暂停执行的函数，或者干脆把协程理解成一个生成器
- 实现：
    - yield返回
    - send调用
- 协程的四个状态：
    - inspect.getgeneratorstate(...)函数确定，该函数会返回下述字符串中的一个：
    - GEN_CREATED: 等待开始执行
    - GEN_RUNNING: 解释器正在执行
    - GEN_SUSPENED: 在yield表达式处暂停
    - GEN_CLOSED: 执行结束
    - next 预激(prime)
- 协程终止
    - 协程中未处理的异常会向上冒泡，传给 next 函数或者 send 方法的调用方(即触发协程的对象）
    - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的 None 和 Ellipsis 等常量经常用作哨符值==。
- yield from
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止时会发出StopIteration异常，异常对象的value属性保存返回值
    - yield from从内部捕获StopIteration异常
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给子生成器
        - 子生成器再把产出的值发送给调用方
        - 子生成器在最后，解释器会抛出StopIteration异常，并且把返回值附加到异常对象上
        

In [26]:
# 协程案例代码1
def simple_coroutine():
    print("-> start")
    x = yield
    print('-> recived', x)
    
# 主线程
sc = simple_coroutine()
print(1111)
# 可以使用sc.send(None), 效果一样
next(sc) # 预激

print(2222)
sc.send("zhaoyun")

1111
-> start
2222
-> recived zhaoyun


StopIteration: 

In [28]:
# 协程代码案例2 协程的状态
def simple_coroutine(a):
    print("-> start")
    
    b = yield a
    print('-> recived', a, b)
    
    c = yield a + b
    print("-> recived", a, b, c)
    
# runc
sc = simple_coroutine(5)
    
aa = next(sc)
print(aa)
bb = sc.send(6) # 5, 6
print(bb)
cc = sc.send(7) # 5, 6, 7
print(cc)


-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [29]:
# 协程代码案例3 yield from
def gen():
    for c in 'AB':
        yield c
        
# list直接用生成器作为参数
print(list(gen()))

def get_new():
    yield from 'AB'
        
print(list(get_new()))

['A', 'B']
['A', 'B']


In [3]:
# 协程代码案例4 委派生成器
'''
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 
表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine）在 
yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为results[key] 赋值的语句还没
有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，
yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''
from collections import namedtuple

ResClass = namedtuple('Res', 'count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count
        
    return ResClass(count, average)

# 委派生成器
def grouper(storages, key):
    while True:
        # 获取verager()返回的值
        storages[key] = yield from averager()
        
# 客户端代码
def client():
    process_data = {
                    'boys_1':[38.2, 42.0, 40.5, 39.8, 39.1, 37.3, 38.5, 37.9, 36.3], 
                    'boys_2':[1.25, 1.28, 1.47, 1.50, 1.35, 1.39, 1.44, 1.26, 1.31],
                    }
    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)
        # 预激协程
        next(coroutine)
        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)
        # 终止协程
        coroutine.send(None)
    print(storages)
    
# run
client()

{'boys_1': Res(count=9, average=38.84444444444444), 'boys_2': Res(count=9, average=1.3611111111111112)}


# asyncio
- python3.4开始引入标准库中，内置对异步io的支持
- asyncio本身是一个消息循环
- 步骤：
    - 创建消息循环
    - 把协程导入
    - 关闭

In [1]:
# 引入异步io包
import asyncio
import threading

# 使用协程
@asyncio.coroutine
def hello():
    print("Hello world! (%s)" % threading.currentThread())
    print("Start...(%s)" % threading.currentThread())
    yield from asyncio.sleep(5)
    print("Done...(%s)" % threading.currentThread())
    print("Hello again! (%s)" % threading.currentThread())
    
# 启动消息循环
loop = asyncio.get_event_loop()
# 定义任务
tasks = [hello(), hello()]
# asyncio使用wait等待tasks执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息循环
loop.close()


Hello world! (<_MainThread(MainThread, started 140019979261696)>)
Start...(<_MainThread(MainThread, started 140019979261696)>)
Hello world! (<_MainThread(MainThread, started 140019979261696)>)
Start...(<_MainThread(MainThread, started 140019979261696)>)
Done...(<_MainThread(MainThread, started 140019979261696)>)
Hello again! (<_MainThread(MainThread, started 140019979261696)>)
Done...(<_MainThread(MainThread, started 140019979261696)>)
Hello again! (<_MainThread(MainThread, started 140019979261696)>)


In [1]:
import asyncio

@asyncio.coroutine
def wget(host):
    print("wget %s..." % host)
    # 异步请求网络地址
    connect = asyncio.open_connection(host, 80)
    # 注意yield from的用法
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        # http协议的换行使用\r\n
        if line == b'\r\n':
            break
        print("%s header > %s" % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()
    
loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()


wget www.sina.com.cn...
wget www.163.com...
wget www.sohu.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Tue, 10 Sep 2019 03:25:26 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 154
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=edge,s=cnc.tianjin.konggang.ha2ts4.755.nb.sinaedge.com,c=1.25.221.168;
www.sina.com.cn header > X-Via-Edge: 1568085926548a8dd19014b3b277d3b50b6aa
www.163.com header > HTTP/1.1 200 OK
www.163.com header > Date: Tue, 10 Sep 2019 03:25:26 GMT
www.163.com header > Content-Type: text/html; charset=GBK
www.163.com header > Connection: close
www.163.com header > Expires: Tue, 10 Sep 2019 03:26:45 GMT
www.163.com header > Server: nginx
www.163.com header > Vary: Accept-Encoding
www.163.com header > Vary: Accept-Encoding
www.163.com header > Cache

# asyncio and await
- 为了更好的表示异步io
- Python3.5引入
- 让协程代码更简洁
- 使用上，可以简单的进行替换
    - 用async替换@asyncio。coroutine
    - await 替换yield from
    

In [1]:
import asyncio
import threading

# @asyncio.coroutine
async def hello():
    print("Hello world! (%s)" % threading.currentThread())
    print("Start...(%s)" % threading.currentThread())
    await asyncio.sleep(5)
    print("Done...(%s)" % threading.currentThread())
    print("Hello again! (%s)" % threading.currentThread())
    
# 启动消息循环
loop = asyncio.get_event_loop()
# 定义任务
tasks = [hello(), hello()]
# asyncio使用wait等待tasks执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息循环
loop.close()

Hello world! (<_MainThread(MainThread, started 140310252082944)>)
Start...(<_MainThread(MainThread, started 140310252082944)>)
Hello world! (<_MainThread(MainThread, started 140310252082944)>)
Start...(<_MainThread(MainThread, started 140310252082944)>)
Done...(<_MainThread(MainThread, started 140310252082944)>)
Hello again! (<_MainThread(MainThread, started 140310252082944)>)
Done...(<_MainThread(MainThread, started 140310252082944)>)
Hello again! (<_MainThread(MainThread, started 140310252082944)>)


# aiohttp
- asyncio实现单线程的并发io，在客户端用处不大
- 在客户端可以asyncio+coroutine配合，因为http是io操作
- asyncio实现了tcp，udp，ssl等协议
- aiohttp是给予asyncio实现的http框架
- pip install aiohttp安装

In [1]:
# aiohttp案例
import asyncio

from aiohttp import web

async def index(request):
    await asyncio.sleep(0.5)
    return web.Response(body=b'<h1>Index</h1>')

async def hello(request):
    await asyncio.sleep(0.5)
    text = '<h1>hello, %s!</h1>' % request.match_info['name']
    return web.Response(body=text.encode('utf-8'))

async def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('GET', '/', index)
    app.router.add_route('GET', '/hello/{name}', hello)
    srv = await loop.create_server(app.make_handler(), '127.0.0.1', 8000)
    print('Server started at http://127.0.0.1:8000...')
    return srv

loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

ModuleNotFoundError: No module named 'aiohttp'

# concurrent.futures
- python3新增的库
- 类似其他语言的线程池的 概念
- 利用multiprocessing实现真正的并行计算
- 核心原理：以子进程的形式，并行运行多个解释器，从而令python程序可以利用多核CPU来提升运行速度。由于子进程与主显示器相分离，所以他们的全局解释器也是相互独立的。每个子进程都能够完整的使用一个CPU内核。
- concurrent.futures.Executor
    - ThreadPoolExcecutor
    - ProcessPoolExecutor
    - 执行的时候需要自行选择
- submit(fn, args, kwargs)
    - fn:异步执行的函数
    - args，kwargs参数
    


In [3]:
# 关于concurrent的案例
from concurrent.futures import ThreadPoolExecutor
import time


def return_future(msg):
    time.sleep(3)
    return msg


# 创建一个线程池
pool = ThreadPoolExecutor(max_workers=2)

# 往线程池加入2个task
f1 = pool.submit(return_future, 'hello')
f2 = pool.submit(return_future, 'world')

# d等待执行完毕
print(f1.done())
time.sleep(3)
print(f2.done())

# 结果
print(f1.result())
print(f2.result())

False
False
hello
world


# current中map函数
- map(fn, \*iterables, timeout=None)
    - 跟map函数相似
    - 函数需要异步执行
    - timeout：超时时间
    - map跟submit使用一个就行
# Future
    - 大拿的同学三分钟后到家做客，大拿想送给朋友一条香烟做礼物
    - 可是大拿手里没有存货
    - 于是让张同学帮大拿去买烟
    - 商店离大拿来回路程30分钟
    - 无奈大拿在朋友来后，只好拿一个空盒子给朋友看，并告诉朋友这是大拿给他的礼物
    - 随后大拿和朋友吹了一个小时牛逼
    - 期间27分钟后张同学带着香烟返回，并将香烟放入烟盒
    - 两个小时后大拿朋友离开，并带走了大拿送给他的香烟
    - 其中用来表示烟盒的就是future

In [5]:
# map案例
import time,re
import os,datetime
from concurrent import futures

data = ['1', '2']

def wait_on(argument):
   print(argument)
   time.sleep(2)
   return "ok"

ex = futures.ThreadPoolExecutor(max_workers=2)
for i in ex.map(wait_on, data):
   print(i)


1
2
ok
ok


In [6]:
# future案例
from concurrent.futures import ThreadPoolExecutor as Pool
from concurrent.futures import as_completed
import requests

URLS = ['http://qq.com', 'http://sina.com', 'http://www.baidu.com', ]


def task(url, timeout=10):
    return requests.get(url, timeout=timeout)


with Pool(max_workers=3) as executor:
    future_tasks = [executor.submit(task, url) for url in URLS]

    for f in future_tasks:
        if f.running():
            print('%s is running' % str(f))

    for f in as_completed(future_tasks):
        try:
            ret = f.done()
            if ret:
                f_ret = f.result()
                print('%s, done, result: %s, %s' % (str(f), f_ret.url, len(f_ret.content)))
        except Exception as e:
            f.cancel()
            print(str(e))

<Future at 0x7fbb627b08d0 state=running> is running
<Future at 0x7fbb62749eb8 state=running> is running
<Future at 0x7fbb6275c4a8 state=running> is running
<Future at 0x7fbb6275c4a8 state=finished returned Response>, done, result: http://www.baidu.com/, 2381
<Future at 0x7fbb62749eb8 state=finished returned Response>, done, result: http://sina.com/, 23994
<Future at 0x7fbb627b08d0 state=finished returned Response>, done, result: https://www.qq.com?fromdefault, 244672
